**Livrable** : Travail de veille et preuve de concept d’une technique récente et conclusion (4 minutes) <br>
Attention, le contenu de ce livrable n'est pas clairement défini.

In [ ]:
# CGVIT TINY

In [4]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay)
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    BatchNormalization,
    LayerNormalization,
    Dense,
    Embedding,
    MultiHeadAttention,
    Layer,
    Add,
    Flatten,
    RandomFlip,
    RandomRotation,
    RandomContrast,
    RandomBrightness
)
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy, TopKCategoricalAccuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import image_dataset_from_directory

In [10]:
# Autres imports
from tqdm import tqdm
import numpy as np 
import pandas as pd
import os
import gc

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output, display
from pprint import pprint

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Model

# Définit le nombre de coeurs utilisés à 10
os.environ['LOKY_MAX_CPU_COUNT'] = '10'

Le dataframe flipkart a été chargé lors du préprocessing dans les globals, réassignons le au nom flipkart pour plus de clarté.

In [14]:
image_df = globals()['flipkart']
image_df.drop(columns=['product_name', 'description'], inplace=True)
print(image_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   image   1050 non-null   object  
 1   class   1050 non-null   category
dtypes: category(1), object(1)
memory usage: 9.7+ KB
None


In [15]:
# Assigne le nombre de classes
n_classes = image_df['class'].nunique()

# Assigne la liste des classes
labels = list(image_df['class'].unique())

# **Séparation des données en train-test-split**

In [26]:
# DEFINITION DES HYPERPARAMETRES DU MODELE
FILENAME_VAR = 'image'
CLASS_VAR = 'class'
# TAILLE DES JEUX DE DONNEES
VAL_SIZE = 0.2
TEST_SIZE = 0.2
# SEED
SEED = 314
BATCH_SIZE = 32
SIZE = 224
CHANNELS = 3
LABELS = list(image_df['class'].unique())
NLABELS = len(LABELS)
EPOCHS = 3
LR = 1e-3
IN_SIZE = 224
PATCH_SIZE = 16
NPATCHES = 256
NHEADS = 4
NLAYERS = 2
HIDDEN_SIZE = 768
DENSE_UNITS = 128


In [24]:
# Création du jeu de données de test et temporaire (train + validation)
X_temp, X_test, y_temp, y_test = train_test_split(
    image_df[FILENAME_VAR], image_df[CLASS_VAR],
    test_size=TEST_SIZE,
    random_state=SEED, stratify=image_df[CLASS_VAR])

# Création du jeu de données de train et de validation
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=VAL_SIZE,
    random_state=SEED, stratify=y_temp)

# Supprime les jeux de données temporaires
del X_temp, y_temp
gc.collect()

# Regroupement des données en dataframe de train et de test
train = pd.concat([X_train, y_train], axis=1)
val = pd.concat([X_val, y_val], axis=1)
test = pd.concat([X_test, y_test], axis=1)

# Supprime les jeux de données temporaires
del X_train, y_train, X_val, y_val, X_test, y_test
gc.collect()

# Affiche les tailles des jeux de données
print(f"Train size: {train.shape[0]}")
print(f"Validation size: {val.shape[0]}")
print(f"Test size: {test.shape[0]}")

Train size: 672
Validation size: 168
Test size: 210


In [75]:
# Model Configs
config = {
    "window_size": (7, 7, 14, 7),
    "embed_dim": 64,
    "depths": (2, 2, 6, 2),
    "num_heads": (2, 4, 8, 16),
    "mlp_ratio": 3.0,
    "path_drop": 0.2,
}
ckpt_link = (
    "https://github.com/awsaf49/gcvit-tf/releases/download/v1.1.6/gcvitxxtiny.keras"
)

# Build Model
model = GCViT(**config)
inp = ops.array(np.random.uniform(size=(1, 224, 224, 3)))
out = model(inp)

# Load Weights
ckpt_path = keras.utils.get_file(ckpt_link.split("/")[-1], ckpt_link)
model.load_weights(ckpt_path)

# Summary
model.summary((224, 224, 3))

NameError: name 'GCViT' is not defined

In [28]:
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess_input

In [33]:
# Fonction pour charger une image à partir du chemin et du label
def preprocess_image_and_load_label(filename, label, SIZE, CHANNELS, preprocessing_func):
    # Vérifie que preprocessing prends une fonction si n'est pas None
    if preprocessing_func is not None and not callable(preprocessing_func):
        raise ValueError("preprocessing doit être une fonction")
    
    # Effectue le prétraitement de l'image
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=CHANNELS)
    image = tf.image.resize(image, [SIZE, SIZE])
    if preprocessing_func is not None:
        try:
            image = preprocessing_func(image)
        except Exception as e:
            print("Vérifiez la fonction de preprocessing")
            raise e
    return image, label

# Fonction pour transformer un dataframe en tf.data.Dataset
def df_to_tfdataset(df, SIZE, BATCH_SIZE, CHANNELS, preprocessing_func=None):
    filenames = df[FILENAME_VAR].values
    labels = df[CLASS_VAR].values
    tfdataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

    # Appliquer la fonction de prétraitement
    tfdataset = tfdataset.map(
        lambda filename, label: preprocess_image_and_load_label(filename, label, SIZE, CHANNELS, preprocessing_func),
        num_parallel_calls=tf.data.experimental.AUTOTUNE
        )
    # Mélanger et mettre en batch
    tfdataset = tfdataset.shuffle(buffer_size=len(df)).batch(BATCH_SIZE)
    return tfdataset

# Shuffle et batcher le dataset
train_dataset = df_to_tfdataset(train, SIZE, BATCH_SIZE, CHANNELS, efficientnet_preprocess_input)
val_dataset = df_to_tfdataset(val, SIZE, BATCH_SIZE, CHANNELS, efficientnet_preprocess_input)
test_dataset = df_to_tfdataset(test, SIZE, BATCH_SIZE, CHANNELS, efficientnet_preprocess_input)


VIT MODEL

In [34]:
from transformers import ViTConfig, ViTModel, AutoImageProcessor, TFViTModel

In [60]:
# Création d'une configuration personnalisée pour le modèle
custom_config = ViTConfig(
    image_size = SIZE,
    hidden_size = HIDDEN_SIZE,
    num_attention_heads = NHEADS,
    num_hidden_layers = NLAYERS
)

# Chargement du modèle pré-entraîné avec la configuration
custom_model = TFViTModel.from_pretrained(
    'google/vit-base-patch16-224-in21k',
    config=custom_config,
    name='vit_custom'
    )

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTModel: ['encoder.layer.2.attention.output.dense.weight', 'encoder.layer.9.intermediate.dense.bias', 'encoder.layer.10.attention.output.dense.weight', 'encoder.layer.4.attention.attention.query.weight', 'encoder.layer.11.attention.attention.value.weight', 'encoder.layer.2.attention.attention.key.bias', 'encoder.layer.6.intermediate.dense.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.8.layernorm_after.bias', 'encoder.layer.9.intermediate.dense.weight', 'encoder.layer.9.output.dense.weight', 'encoder.layer.6.layernorm_before.bias', 'encoder.layer.9.layernorm_after.bias', 'encoder.layer.5.layernorm_after.weight', 'encoder.layer.8.layernorm_before.weight', 'encoder.layer.11.intermediate.dense.bias', 'encoder.layer.2.attention.attention.key.weight', 'encoder.layer.3.attention.attention.key.bias', 'encoder.layer.6.attention.attention.query.bias', 'encoder.layer.7.attention.attention.k

In [61]:
# Chargement du modèle pré-entraîné avec la configuration
base_model = TFViTModel.from_pretrained(
    'google/vit-base-patch16-224-in21k',
    name='vit'
    )

All PyTorch model weights were used when initializing TFViTModel.

All the weights of TFViTModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


In [41]:
base_model.config

ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224-in21k",
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.44.0"
}

In [59]:
from tensorflow.keras.layers import(
    Conv2D,
    BatchNormalization,
    LayerNormalization,
    Dense,
    Input,
    Embedding,
    MultiHeadAttention,
    Layer,
    Add,
    Resizing,
    Rescaling,
    Permute,
    Flatten,
    RandomFlip,
    RandomRotation,
    RandomContrast,
    RandomBrightness
)

In [65]:
data_preprocessing = tf.keras.Sequential([
    Resizing(SIZE, SIZE),
    Rescaling(1./255),
    Permute((3, 1, 2)) # Permute les dimensions en (3, 224, 224)
])

In [66]:
data_augmentation = tf.keras.Sequential([
    RandomFlip("horizontal_and_vertical"),
    RandomRotation(0.2),
    RandomContrast(0.2),
    RandomBrightness(0.2)
])

In [68]:
inputs = Input(shape=(SIZE, SIZE, CHANNELS))
base_model.trainable = False
x = base_model(inputs, training=False)[0]
outputs = Dense(NLABELS, activation='softmax')(x)


ValueError: Exception encountered when calling layer 'vit' (type TFViTModel).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for pixel_values.

Call arguments received by layer 'vit' (type TFViTModel):
  • pixel_values=<KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=None, name=keras_tensor_34>
  • head_mask=None
  • output_attentions=None
  • output_hidden_states=None
  • interpolate_pos_encoding=None
  • return_dict=None
  • training=False

In [69]:
# Select an Image Classification model

model_name = "vit-b8"

model_handle_map = {
  "vit-b8": "/kaggle/input/vision-transformer/tensorflow2/vit-b8-classification/1",
#   "evit-b8": "/kaggle/input/vision-transformer/tensorflow2/vit-b8-classification/1",
#   "vit-b8": "/kaggle/input/efficientnet-v2/tensorflow2/imagenet1k-b0-classification/versions/1",
#   "vit-b8": "https://kaggle.com/models/kaggle/vision-transformer/frameworks/TensorFlow2/variations/vit-b8-classification/",
#   "vit-b8": "https://kaggle.com/models/kaggle/vision-transformer/frameworks/TensorFlow2/variations/vit-b8-classification/versions/1",
#   "vit-b8": "https://tfhub.dev/sayakpaul/vit_b8_classification/1",
}


model_image_size_map = {
  "vit-b8": 224,
}

model_handle = model_handle_map[model_name]

print(f"Selected model: {model_name} : {model_handle}")

Selected model: vit-b8 : /kaggle/input/vision-transformer/tensorflow2/vit-b8-classification/1


In [70]:
model_image_size_map = {
  "vit-b8": 224,
}

model_handle = model_handle_map.get(model_name)
pixels = model_image_size_map.get(model_name, 224)

print(f"Selected model: {model_name} : {model_handle}")

IMAGE_SIZE = (pixels, pixels)
print(f"Input size {IMAGE_SIZE}")

BATCH_SIZE = 16#@param {type:"integer"}

Selected model: vit-b8 : /kaggle/input/vision-transformer/tensorflow2/vit-b8-classification/1
Input size (224, 224)


In [72]:
# import tensorflow_hub as hub
import tensorflow_hub as hub

In [73]:
labels

['Home Furnishing', 'Baby Care', 'Baby Care', 'Home Furnishing', 'Home Furnishing', ..., 'Baby Care', 'Baby Care', 'Baby Care', 'Baby Care', 'Baby Care']
Length: 1050
Categories (7, object): ['Baby Care', 'Beauty and Personal Care', 'Computers', 'Home Decor & Festive Needs', 'Home Furnishing', 'Kitchen & Dining', 'Watches']

In [74]:
do_fine_tuning = False 

print("Building model with", model_handle)
model = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(model_handle, trainable=do_fine_tuning),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(len(labels),
                          kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
model.build((None,)+IMAGE_SIZE+(3,))
model.summary()

Building model with /kaggle/input/vision-transformer/tensorflow2/vit-b8-classification/1



c:\Users\BS6517\Documents_local\projet_6\oc_projet_6\.venv\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


OSError: /kaggle/input/vision-transformer/tensorflow2/vit-b8-classification/1 does not exist.